In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus


user = 'root'
password = "YOUR_PASSWORD"
host = 'localhost'
port = '3306'
db = 'bank_marketing'

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}")

query = "SELECT * FROM bank_additional_staging2"

In [ ]:
use_sql = False

if use_sql:
    df = pd.read_sql(query, engine)
else:
    df = pd.read_csv('data/bank_marketing_cleaned.csv')

In [131]:
df.shape

(34979, 25)

In [132]:
df.describe()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,financial_stress,unknown_count,contact_intensity,economic_index
count,34979.000000,34979.000000,34979.000000,34979.000000,34979.000000,34979.000000,34979.000000,34979.000000,34979.000000,34979.000000,34979.000000,34979.000000,34979.000000,34979.000000
mean,39.767003,255.692044,2.667715,993.009720,0.088710,0.507522,93.656955,-40.961551,4.094648,5189.495440,0.671660,0.286543,2.756425,-36.359382
std,9.355824,260.575357,2.922361,76.896209,0.299127,1.263823,0.503603,3.972207,1.429429,49.003174,0.631977,0.526907,2.917680,6.229003
min,18.000000,0.000000,1.000000,0.000000,0.000000,-1.800000,92.756000,-50.000000,1.250000,5099.000000,0.000000,0.000000,1.000000,-50.290000
25%,32.000000,100.000000,1.000000,999.000000,0.000000,-0.100000,93.200000,-42.700000,4.076000,5191.000000,0.000000,0.000000,1.000000,-38.024000
50%,38.000000,175.000000,2.000000,999.000000,0.000000,1.100000,93.918000,-42.000000,4.860000,5196.000000,1.000000,0.000000,2.000000,-36.332000
75%,47.000000,314.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.962000,5228.000000,1.000000,1.000000,3.000000,-30.443000
max,95.000000,4918.000000,56.000000,999.000000,3.000000,1.400000,94.465000,-36.100000,5.045000,5228.000000,2.000000,3.000000,56.000000,-29.730000


In [133]:
df['y'].value_counts()  # HIGHLY IMBALANCED

y
no     32717
yes     2262
Name: count, dtype: int64

In [134]:
X = df.drop(['y','duration'], axis=1)
y = df['y']

In [135]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [136]:
from sklearn.preprocessing import LabelEncoder

target_encoder = LabelEncoder()

y_train_encoded = target_encoder.fit_transform(y_train)
y_test_encoded = target_encoder.transform(y_test)

In [137]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector

month_cat = [['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']]
day_cat = [['sat', 'sun', 'mon', 'tue', 'wed', 'thu', 'fri']]

month_encoder = OneHotEncoder(categories=month_cat, handle_unknown='ignore')
day_encoder = OneHotEncoder(categories=day_cat, handle_unknown='ignore')
cat_encoder = OneHotEncoder(handle_unknown='ignore')


categorical_transformer = ColumnTransformer(
    transformers = [
        ('month_ohe', month_encoder, ['month']),
        ('day_ohe', day_encoder, ['day_of_week']),
        ('cat_ohe', cat_encoder, make_column_selector(dtype_include='object'))

    ],
    remainder='passthrough'
)



# Gradient Boosting Pipeline

In [196]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

pipe_rf = Pipeline([
         ('encoding', categorical_transformer),
         ('classifier', RandomForestClassifier(
                                            n_estimators=100,
                                            max_depth = 10,
                                            min_samples_leaf = 30,
                                            class_weight='balanced', 
                                            random_state=42,
                                            ))
])

pipe_rf.fit(X_train, y_train_encoded)



,steps,"[('encoding', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('month_ohe', ...), ('day_ohe', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [139]:
from sklearn.inspection import permutation_importance

result_rf = permutation_importance(pipe_rf, X_test, y_test_encoded, scoring='f1', n_repeats=10, random_state=42)

for i, col in enumerate(X.columns):
    print(f"{col}: {result_rf.importances_mean[i]:.4f} ± {result_rf.importances_std[i]:.4f}")


age: 0.0209 ± 0.0065
job: 0.0053 ± 0.0083
marital: 0.0188 ± 0.0040
education: 0.0209 ± 0.0120
default: 0.0194 ± 0.0063
housing: -0.0064 ± 0.0106
loan: 0.0000 ± 0.0034
contact: 0.0474 ± 0.0127
month: 0.1081 ± 0.0055
day_of_week: 0.0190 ± 0.0070
campaign: 0.0064 ± 0.0056
pdays: -0.0000 ± 0.0001
previous: -0.0024 ± 0.0023
poutcome: -0.0057 ± 0.0035
emp.var.rate: 0.0335 ± 0.0051
cons.price.idx: 0.0262 ± 0.0038
cons.conf.idx: 0.0422 ± 0.0060
euribor3m: 0.0509 ± 0.0069
nr.employed: 0.0286 ± 0.0072
financial_stress: -0.0062 ± 0.0048
unknown_count: 0.0063 ± 0.0042
contact_intensity: 0.0084 ± 0.0071
economic_index: 0.0810 ± 0.0064


In [197]:
imporatant_features = ['month', 'economic_index', 'emp.var.rate' , 'nr.employed','contact','age', 'education', 'day_of_week']

X_train_final = X_train[imporatant_features]
X_test_final = X_test[imporatant_features]

pipe_rf.fit(X_train_final, y_train_encoded)

,steps,"[('encoding', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('month_ohe', ...), ('day_ohe', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [198]:
from sklearn.metrics import accuracy_score, classification_report


y_pred_test = pipe_rf.predict(X_test_final)
y_pred_train = pipe_rf.predict(X_train_final)

print("Accuracy Train:", accuracy_score(y_train_encoded, y_pred_train))
print("Accuracy Test:", accuracy_score(y_test_encoded, y_pred_test))

print(classification_report(y_test_encoded, y_pred_test))

Accuracy Train: 0.7634635314297966
Accuracy Test: 0.7505717552887364
              precision    recall  f1-score   support

           0       0.95      0.77      0.85      6528
           1       0.13      0.49      0.21       468

    accuracy                           0.75      6996
   macro avg       0.54      0.63      0.53      6996
weighted avg       0.90      0.75      0.81      6996



# LightGBM Pipeline

In [199]:
from lightgbm import LGBMClassifier

pipe_lgb = Pipeline([
         ('classifier', LGBMClassifier(random_state=42))
])

cat_cols = [
    'job', 'marital', 'education', 'default',
    'housing', 'loan', 'contact',
    'month', 'day_of_week', 'poutcome'
]

for col in cat_cols:
    X_train[col] = X_train[col].astype('category')
    X_test[col] = X_test[col].astype('category')

pipe_lgb.fit(X_train, y_train_encoded)

[LightGBM] [Info] Number of positive: 1794, number of negative: 26189
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001289 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 462
[LightGBM] [Info] Number of data points in the train set: 27983, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.064110 -> initscore=-2.680892
[LightGBM] [Info] Start training from score -2.680892


,steps,"[('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.1
,n_estimators,100
,subsample_for_bin,200000
,objective,None


In [200]:
result_lgb = permutation_importance(pipe_lgb, X_test, y_test_encoded, scoring='f1', n_repeats=10, random_state=42)

for i, col in enumerate(X.columns):
    print(f"{col}: {result_lgb.importances_mean[i]:.4f} ± {result_lgb.importances_std[i]:.4f}")

age: 0.0262 ± 0.0099
job: 0.0135 ± 0.0056
marital: 0.0011 ± 0.0039
education: 0.0091 ± 0.0050
default: 0.0157 ± 0.0088
housing: 0.0044 ± 0.0049
loan: 0.0015 ± 0.0018
contact: 0.0284 ± 0.0063
month: 0.0540 ± 0.0054
day_of_week: 0.0128 ± 0.0074
campaign: 0.0107 ± 0.0091
pdays: 0.0215 ± 0.0011
previous: 0.0051 ± 0.0041
poutcome: -0.0088 ± 0.0036
emp.var.rate: -0.0115 ± 0.0046
cons.price.idx: 0.0080 ± 0.0032
cons.conf.idx: 0.0164 ± 0.0056
euribor3m: -0.0158 ± 0.0109
nr.employed: 0.0000 ± 0.0000
financial_stress: 0.0040 ± 0.0026
unknown_count: 0.0105 ± 0.0061
contact_intensity: 0.0053 ± 0.0055
economic_index: 0.0592 ± 0.0064


In [201]:
X_train_final[['month','education', 'day_of_week','contact']] = X_train_final[['month','education', 'day_of_week', 'contact']].astype('category')
X_test_final[['month','education', 'day_of_week', 'contact']] = X_test_final[['month','education', 'day_of_week', 'contact']].astype('category')

pipe_lgb.fit(X_train_final, y_train_encoded)


[LightGBM] [Info] Number of positive: 1794, number of negative: 26189
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000596 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 210
[LightGBM] [Info] Number of data points in the train set: 27983, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.064110 -> initscore=-2.680892
[LightGBM] [Info] Start training from score -2.680892


C:\Users\Hp\AppData\Local\Temp\ipykernel_20700\2390305713.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_final[['month','education', 'day_of_week','contact']] = X_train_final[['month','education', 'day_of_week', 'contact']].astype('category')
C:\Users\Hp\AppData\Local\Temp\ipykernel_20700\2390305713.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_final[['month','education', 'day_of_week', 'contact']] = X_test_final[['month','education', 'day_of_week', 'contact']].astype('category')


,steps,"[('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.1
,n_estimators,100
,subsample_for_bin,200000
,objective,None


In [202]:
y_pred_test = pipe_lgb.predict(X_test_final)
y_pred_train = pipe_lgb.predict(X_train_final)


print("Accuracy Train:", accuracy_score(y_train_encoded, y_pred_train))
print("Accuracy Test:", accuracy_score(y_test_encoded, y_pred_test))

print(classification_report(y_test_encoded, y_pred_test))

Accuracy Train: 0.9404281170710789
Accuracy Test: 0.9328187535734705
              precision    recall  f1-score   support

           0       0.94      0.99      0.97      6528
           1       0.49      0.08      0.13       468

    accuracy                           0.93      6996
   macro avg       0.71      0.54      0.55      6996
weighted avg       0.91      0.93      0.91      6996

